In [11]:
# !pip install gymnasium
# !pip install "gymnasium[other]"

# Start a frozenlake instance, and find where the holes are

In [12]:
import numpy as np
import gymnasium as gym

def find_holes(env):
    """
    Identify the positions of holes in the Frozen Lake environment.
    """
    desc = env.unwrapped.desc  # Access the environment's map
    holes = []
    for row in range(desc.shape[0]):
        for col in range(desc.shape[1]):
            if desc[row, col] == b'H':  # 'H' represents a hole
                holes.append((row, col))
    return holes

# Create the Frozen Lake environment
env = gym.make("FrozenLake-v1", is_slippery=False, render_mode=None)
env.reset()

# Find and print the holes
holes = find_holes(env)
print("Holes are located at:", holes)


Holes are located at: [(1, 1), (1, 3), (2, 3), (3, 0)]


# Download and install Metric-FF, a PDDL solver

Download MetricFF: 
https://github.com/tatsubori/Metric-FF worked for me
https://fai.cs.uni-saarland.de/hoffmann/metric-ff.html (offical) didn’t
Navigate to folder and run `make`
then run `./ff`

is meant for Mac or Linux, but github has a cross platform version: https://github.com/Vidminas/metric-ff-crossplatform

# After Creating PDDL domain and problem file, run metric-FF to solve it 

usage of ff:

OPTIONS   DESCRIPTIONS

-p <str>    Path for operator and fact file
-o <str>    Operator file name
-f <str>    Fact file name

-r <int>    Random seed [used for random restarts; preset: 0]

-s <int>    Search configuration [preset: s=5]; '+H': helpful actions pruning
      0     Standard-FF: EHC+H then BFS (cost minimization: NO)
      1     BFS (cost minimization: NO)
      2     BFS+H (cost minimization: NO)
      3     Weighted A* (cost minimization: YES)
      4     A*epsilon (cost minimization: YES)
      5     EHC+H then A*epsilon (cost minimization: YES)
-w <num>    Set weight w for search configs 3,4,5 [preset: w=5]

-C          Do NOT use cost-minimizing relaxed plans for options 3,4,5

-b <float>  Fixed upper bound on solution cost (prune based on g+hmax); active only with cost minimization

In [3]:
import subprocess
import os

# Paths to PDDL files
domain_file = "./fl_domain.pddl"
problem_file = "./fl_prob.pddl"

def run_metricff(domain_file, problem_file):
    # First, check if files exist
    if not os.path.exists(domain_file):
        raise FileNotFoundError(f"Domain file not found: {domain_file}")
    if not os.path.exists(problem_file):
        raise FileNotFoundError(f"Problem file not found: {problem_file}")
    
    # Print file contents for debugging
    print("Domain file contents:")
    with open(domain_file, 'r') as f:
        print(f.read())
    print("\nProblem file contents:")
    with open(problem_file, 'r') as f:
        print(f.read())
    
    # Path to MetricFF binary
    metricff_path = "./Metric-FF-master/ff"
    if not os.path.exists(metricff_path):
        raise FileNotFoundError(f"MetricFF executable not found: {metricff_path}")
    
    # Make sure ff has execute permissions
    os.chmod(metricff_path, 0o755)
    
    cmd = [metricff_path, "-o", domain_file, "-f", problem_file, "-s", "0"]
    print("\nExecuting command:", ' '.join(cmd))
    
    try:
        # Execute MetricFF and capture output
        result = subprocess.run(cmd, capture_output=True, text=True, check=True)
        
        print("\nMetricFF stdout:")
        print(result.stdout)
        
        # Parse the plan from MetricFF's output
        plan = []
        capture_plan = False
        for line in result.stdout.splitlines():
            line = line.strip()
            if line.lower().startswith("step"):  # Start capturing actions
                capture_plan = True
            if capture_plan and ":" in line:  # Parse actions after "step" starts
                action = line.split(":")[1].strip().lower()
                plan.append(action)
            if line == "":  # Stop capturing on blank lines
                capture_plan = False
        
        return plan
        
    except subprocess.CalledProcessError as e:
        print("\nMetricFF stderr:")
        print(e.stderr)
        print("\nMetricFF stdout:")
        print(e.stdout)
        raise RuntimeError(f"MetricFF failed with return code {e.returncode}")
    except Exception as e:
        print(f"Unexpected error: {str(e)}")
        raise

try:
    # Run MetricFF and get the plan
    plan = run_metricff(domain_file, problem_file)
    print("\nPlan generated by MetricFF:", plan)
except Exception as e:
    print(f"Error: {str(e)}")

Domain file contents:
(define (domain frozenlake)
  (:requirements :strips :typing)
  (:types x y)
  (:predicates
    (at ?x - x ?y - y)     ; the agent is at position (x, y)
    (goal ?x - x ?y - y)    ; the goal is at position (x, y)
    (hole ?x - x ?y - y)    ; there is a hole at position (x, y)
    (free ?x - x ?y - y)    ; the cell is free and not a hole
    (next_x ?p1 ?p2 - x)    ; p2 is next to p1 on x axis
    (next_y ?p1 ?p2 - y))   ; p2 is next to p1 on y axis

  (:action move-down
    :parameters (?x1 - x ?x2 - x ?y - y)
    :precondition (and
      (at ?x1 ?y)
      (next_x ?x1 ?x2)
      (free ?x2 ?y))
    :effect (and
      (not (at ?x1 ?y))
      (at ?x2 ?y)))

  (:action move-up
    :parameters (?x1 - x ?x2 - x ?y - y)
    :precondition (and
      (at ?x1 ?y)
      (next_x ?x2 ?x1)
      (free ?x2 ?y))
    :effect (and
      (not (at ?x1 ?y))
      (at ?x2 ?y)))

  (:action move-right
    :parameters (?x - x ?y1 - y ?y2 - y)
    :precondition (and
      (at ?x ?y1)
  

# Convert pddl actions to gym actions

In [4]:
plan

['move-down pos_x_0 pos_x_1 pos_y_0',
 'move-down pos_x_1 pos_x_2 pos_y_0',
 'move-right pos_x_2 pos_y_0 pos_y_1',
 'move-down pos_x_2 pos_x_3 pos_y_1',
 'move-right pos_x_3 pos_y_1 pos_y_2',
 'move-right pos_x_3 pos_y_2 pos_y_3']

In [5]:
def convert_plan_to_actions(plan):
    action_mapping = {
        'move-left': 0,
        'move-down': 1,
        'move-right': 2,
        'move-up': 3
    }
    actions = [action_mapping[step.split()[0]] for step in plan]
    return actions

convert_plan_to_actions(plan)

[1, 1, 2, 1, 2, 2]

# Run plan in the frozen lake env

In [7]:
import gymnasium as gym

def execute_plan_in_frozenlake(actions):
    env = gym.make("FrozenLake-v1", is_slippery=False, render_mode="human")
    obs = env.reset()
    done = False
    step_count = 0
    
    for action in actions:
        if done:
            break
        obs, reward, done, t, info = env.step(action)
        step_count += 1
    
    env.close()
    return step_count, reward

# Convert the plan to actions
actions = convert_plan_to_actions(plan)
steps, final_reward = execute_plan_in_frozenlake(actions)
print(f"Execution completed in {steps} steps. Final reward: {final_reward}")


Execution completed in 6 steps. Final reward: 1.0


In [15]:
import gymnasium as gym
from gymnasium.wrappers import RecordVideo

def execute_plan_in_frozenlake(actions, video_dir="recordings"):
    # Create the environment with video recording
    env = RecordVideo(
        gym.make("FrozenLake-v1", is_slippery=False, render_mode="rgb_array"),
        video_folder=video_dir,
        disable_logger=True  # Suppresses Gym logging output
    )
    obs, info = env.reset()
    done = False
    step_count = 0

    for action in actions:
        if done:
            break
        obs, reward, done, t, info = env.step(action)
        step_count += 1
    
    env.close()
    return step_count, reward

actions = convert_plan_to_actions(plan)

# Execute the plan and record video
steps, final_reward = execute_plan_in_frozenlake(actions, video_dir="frozenlake_videos")
print(f"Execution completed in {steps} steps. Final reward: {final_reward}")


Execution completed in 6 steps. Final reward: 1.0
